### **F12 개발자도구 console에 입력**
#### **하나의 cell은 최대 12시간까지만 실행 가능 + 90분 동안 사용자 움직임 없으면 멈춤**

In [ ]:
"""
function ClickConnect() {
  console.log("10분마다 자동 재연결");
  document.querySelector("colab-toolbar-button#connect").click();
}
setInterval(ClickConnect,1000*60*10);
"""

### **뉴스 & 댓글 텍스트를 교정하기 위한 라이브러리 설치**
설치 후 런타임 다시 실행해야 하는 경우 존재

#### soynlp
https://github.com/lovit/soynlp
#### PyKoSpacing
https://github.com/haven-jeon/PyKoSpacing
#### py-hanspell
https://github.com/ssut/py-hanspell

In [ ]:
!pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 416 kB 6.7 MB/s 


In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-u3jepulg
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-u3jepulg
     - 570.3 MB 61.2 MB/s
     |████████████████████████████████| 14.8 MB 7.7 MB/s 
     |████████████████████████████████| 1.3 MB 48.2 MB/s 
     |████████████████████████████████| 132 kB 48.6 MB/s 
     |████████████████████████████████| 462 kB 58.0 MB/s 
     |████████████████████████████████| 5.6 MB 25.3 MB/s 
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2268580 sha256=fd898cab788a66458ae68c98f6373aadbf157773c47c94abbbc9d6f439459e8f
  Stored in directory: /tmp/pip-ephem-wheel-cache-we_6gfpn/wheels/9b/93/81/a2a7dc8c66ede5bf30634d20635f32b95eac7ca2ea8844058b
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30694 sha256=b6aa051faa9bacdd2f810294de

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-s8lu50wn
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-s8lu50wn
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=65990fb1648af1122b437445ae197c3c89f6f59e06ecfbba6d6fd32788602337
  Stored in directory: /tmp/pip-ephem-wheel-cache-veg0fbsj/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


### **클래스 정의 & 선언**

In [ ]:
import numpy as np
import pandas as pd
from soynlp.normalizer import *
from pykospacing import Spacing
from hanspell import spell_checker

In [ ]:
class CorrectNews:
  NewsDetail = pd.read_csv('./Raw/news.csv', header = None, encoding="utf-8")
  NewsDetail.columns = ['date','title','summary']
  Comment = pd.DataFrame(columns = ['comment'])

  NameList = ['20211227.csv','20211228.csv','20211229.csv','20211230.csv','20211231.csv']
 #'20211226.csv','20211227.csv','20211228.csv','20211229.csv','20211230.csv','20211231.csv'
 
  len_news = 0
  len_comment = 0

  spacing = Spacing()

  def initNews(self):
    self.NewsDetail = self.NewsDetail.fillna("Null")
    self.len_news = len(self.NewsDetail)

  def initComment(self, path):
    tmp = pd.read_csv(path, header = None, encoding="utf-8")
    tmp.columns = ['comment']
    self.Comment = tmp.fillna("Null")
    self.len_comment = len(self.Comment)

  # 한글, 숫자, 영어, 문장부호만 남기고 삭제
  def deleteSymbol(self, flag):
    if flag == 'news':
      for i in range(self.len_news):
        tmp_date = self.NewsDetail['date'].iloc[i]
        tmp_title = only_text(self.NewsDetail['title'].iloc[i])
        tmp_summary = only_text(self.NewsDetail['summary'].iloc[i])
        self.NewsDetail.loc[i] = [tmp_date, tmp_title, tmp_summary]
    elif flag == 'comment':
      for i in range(self.len_comment):
        tmp_comment = only_text(self.Comment['comment'].iloc[i])
        self.Comment.loc[i] = tmp_comment
    else:
      print("flag 입력 없음")

  # 띄어쓰기 교정
  def correctSpacing(self, flag):
    if flag == 'news':
      for i in range(self.len_news):
        tmp_date = self.NewsDetail['date'].iloc[i]
        tmp_title = self.spacing(self.NewsDetail['title'].iloc[i])
        tmp_summary = self.spacing(self.NewsDetail['summary'].iloc[i])
        self.NewsDetail.loc[i] = [tmp_date, tmp_title, tmp_summary]
    elif flag == 'comment':
      for i in range(self.len_comment):
        tmp_comment = self.spacing(self.Comment['comment'].iloc[i])
        self.Comment.loc[i] = tmp_comment
    else:
      print("flag 입력 없음")

  # 맞춤법 교정
  def correctSpelling(self, flag):
    if flag == 'news':
      for i in range(self.len_news):
        print(i,"번째")
        # date
        tmp_date = self.NewsDetail['date'].iloc[i]
        # title
        tmp_str = self.NewsDetail['title'].iloc[i].encode("UTF-8")
        result = spell_checker.check(tmp_str)
        tmp_title = result.checked
        # summary
        tmp_str = self.NewsDetail['summary'].iloc[i].encode("UTF-8")
        result = spell_checker.check(tmp_str)
        tmp_summary = result.checked
        self.NewsDetail.loc[i] = [tmp_date, tmp_title, tmp_summary]
    elif flag == 'comment':
      for i in range(self.len_comment):
        print(i,"번째")
        # comment
        tmp_str = self.Comment['comment'].iloc[i].encode("UTF-8")
        result = spell_checker.check(tmp_str)
        self.Comment.loc[i] = result.checked
    else:
      print("flag 입력 없음")

  # 반복되는 자모 제거
  def deleteAlphabet(self, flag):
    if flag == 'news':
      for i in range(self.len_news):
        tmp_date = self.NewsDetail['date'].iloc[i]
        tmp_title = repeat_normalize(self.NewsDetail['title'].iloc[i], num_repeats = 2)
        tmp_summary = repeat_normalize(self.NewsDetail['summary'].iloc[i], num_repeats = 2)
        self.NewsDetail.loc[i] = [tmp_date, tmp_title, tmp_summary]
    elif flag == 'comment':
      for i in range(self.len_comment):
        tmp_comment = repeat_normalize(self.Comment['comment'].iloc[i], num_repeats = 2)
        self.Comment.loc[i] = tmp_comment
    else:
      print("flag 입력 없음")

  # news.csv 저장
  def storeNews(self):
    self.NewsDetail.to_csv('./Corrected/news.csv', header = None, index = False, encoding="utf-8-sig")
  # yyyymmdd.csv 저장
  def storeComment(self, path):
    self.Comment.to_csv(path, header = None, index = False, encoding="utf-8-sig")

In [ ]:
cor = CorrectNews()

### **뉴스 텍스트 교정**

In [ ]:
# 한글, 영어, 숫자, 문장부호만 남기고 삭제
cor.initNews()
print(cor.len_news)
cor.deleteSymbol('news')
cor.NewsDetail

In [ ]:
# 띄어쓰기 교정
cor.correctSpacing('news')
cor.NewsDetail

In [ ]:
# 맞춤법 교정
cor.correctSpelling('news')
cor.NewsDetail

In [ ]:
# 반복되는 자모 삭제
cor.deleteAlphabet('news')
cor.NewsDetail

### **뉴스 텍스트 저장**

In [ ]:
cor.storeNews()

### **댓글 텍스트 교정 & 저장**

반복문 ver.

```
for path in cor.NameList:
  print(path)
  selected = './Raw/' + path   # load data
  corrected = './Corrected/' + path # store data
  cor.initComment(selected)
  print(cor.len_comment)
  cor.deleteSymbol('comment')
  print("한글, 영어, 숫자, 문장부호만 남기고 삭제")
  cor.correctSpacing('comment')
  print("띄어쓰기 교정")
  cor.correctSpelling('comment')
  print("맞춤법 교정")
  cor.deleteAlphabet('comment')
  print("반복되는 자모 삭제")
  cor.storeComment(corrected)
  print("저장")
```

In [ ]:
path = '20211201.csv'
print(path)
selected = './Raw/' + path   # load data
corrected = './Corrected/' + path # store data
cor.initComment(selected)
print(cor.len_comment)
cor.deleteSymbol('comment')
print("한글, 영어, 숫자, 문장부호만 남기고 삭제 완료")
cor.correctSpacing('comment')
print("띄어쓰기 교정 완료")
cor.correctSpelling('comment')
print("맞춤법 교정 완료")
cor.deleteAlphabet('comment')
print("반복되는 자모 삭제 완료")
cor.storeComment(corrected)
print("저장")

### **에러 발생**

In [ ]:
path = '20211227.csv'
print(path)
selected = './Raw/' + path   # load data
corrected = './Corrected/' + path # store data
cor.initComment(selected)
print(cor.len_comment)

20211220-20083.csv
120848


In [ ]:
# 댓글
# 에러 발생했을 경우, 발생한 행부터 이어서 하기
for i in range(115224, cor.len_comment):
        print(i,"번째")
        tmp_str = cor.Comment['comment'].iloc[i].encode("UTF-8")
        result = spell_checker.check(tmp_str)
        cor.Comment.loc[i] = result.checked

cor.deleteAlphabet('comment')
print("반복되는 자모 삭제 완료")
cor.storeComment(corrected)
print("저장")

In [ ]:
# 뉴스
# 에러 발생했을 경우 이어서 하기
for i in range(9524, cor.len_news):
        print(i,"번째")
        # date
        tmp_date = cor.NewsDetail['date'].iloc[i]
        # title
        tmp_str = cor.NewsDetail['title'].iloc[i].encode("UTF-8")
        result = spell_checker.check(tmp_str)
        tmp_title = result.checked
         # summary
        tmp_str = cor.NewsDetail['summary'].iloc[i].encode("UTF-8")
        result = spell_checker.check(tmp_str)
        tmp_summary = result.checked
        cor.NewsDetail.loc[i] = [tmp_date, tmp_title, tmp_summary]

cor.deleteAlphabet('news')
print("반복되는 자모 삭제 완료")
cor.storeNews(corrected)
print("저장")